In [31]:
import pandas as pd
import numpy as np

In [32]:
keywords = pd.DataFrame.from_dict({})

In [33]:
keywords

""


In [34]:
def add_column_if_missing(name, df):
    if not name in df.columns:
        df[name] = False

In [35]:
def add_row(index, df):
    df.loc[index,:] = False

In [36]:
def get_index(df_or_s, default=None):
    index = df_or_s.last_valid_index()
    if index is None:
        return 0
    else:
        return index + 1

In [37]:
def add_keywords(keyword_list, df):
    index = get_index(df, default=False)
    for keyword in keyword_list:
        add_column_if_missing(keyword, df)
    df.loc[index,keyword_list] = True
    other_columns = [column for column in df.columns if column not in keyword_list]
    df.loc[index, other_columns] = False
    return index

In [38]:
SAMPLE_KEYWORDS = [
    ['depression', 'Champlain'],
    ['depression', 'Mauricie'],
    ['travail', 'Champlain'],
    ['humeur', 'TDAH', 'Shawinigan'],
    ['depression', 'travail', 'Shawinigan']
]

In [39]:
for keyword_list in SAMPLE_KEYWORDS:
    add_keywords(keyword_list, keywords)

In [40]:
keywords

,depression,Champlain,Mauricie,travail,humeur,TDAH,Shawinigan
0,True,True,False,False,False,False,False
1,True,False,True,False,False,False,False
2,False,True,False,True,False,False,False
3,False,False,False,False,True,True,True
4,True,False,False,True,False,False,True


In [42]:
keywords.loc[keywords.any(axis=1),:]

,depression,Champlain,Mauricie,travail,humeur,TDAH,Shawinigan
0,True,True,False,False,False,False,False
1,True,False,True,False,False,False,False
2,False,True,False,True,False,False,False
3,False,False,False,False,True,True,True
4,True,False,False,True,False,False,True


In [43]:
def delete_match(db, key1, key2):
    """ Delete match from database and unused rows and columns | str, str -> None """
    db.loc[key1, key2] = False
    db = db.loc[:, db.any()]
    db = db.loc[db.any(axis=1),:]
    return db

In [51]:
keywords = delete_match(keywords, 0, 'Champlain')

In [52]:
keywords

,Champlain,Mauricie,travail,humeur,TDAH,Shawinigan
1,False,True,False,False,False,False
2,True,False,True,False,False,False
3,False,False,False,True,True,True
4,False,False,True,False,False,True


In [11]:
texts = pd.Series([], dtype='object')

In [12]:
def add_text(text, s, index=None):
    if not index:
        index = get_index(s, default='')
    s.loc[index] = text

In [13]:
SAMPLE_TEXTS = [
    'Je recommande telle thérapie de telle durée avec telles modalités',
    'Voici ma deuxieme recommandation',
    'Le client aurait besoin de tant de séances portant sur tel enjeu',
    'Je recommande un suivi sur les capacités parentales',
    'Une évaluation au domicile serait nécessaire pour poursuivre le processus'    
]

In [14]:
for text in SAMPLE_TEXTS:
    add_text(text, texts)

In [15]:
texts

0    Je recommande telle thérapie de telle durée av...
1                     Voici ma deuxieme recommandation
2    Le client aurait besoin de tant de séances por...
3    Je recommande un suivi sur les capacités paren...
4    Une évaluation au domicile serait nécessaire p...
dtype: object

In [16]:
def add_entry(text, keyword_list, text_series, keyword_dataframe):
    index = add_keywords(keyword_list, keyword_dataframe)
    add_text(text, text_series, index=index)

In [17]:
keywords = pd.DataFrame.from_dict({})
texts = pd.Series([], dtype='object')

In [18]:
texts

Series([], dtype: object)

In [19]:
for keyword_list, text in zip(SAMPLE_KEYWORDS, SAMPLE_TEXTS):
    add_entry(text, keyword_list, texts, keywords)

In [20]:
keywords

,depression,Champlain,Mauricie,travail,humeur,TDAH,Shawinigan
0,True,True,False,False,False,False,False
1,True,False,True,False,False,False,False
2,False,True,False,True,False,False,False
3,False,False,False,False,True,True,True
4,True,False,False,True,False,False,True


In [21]:
texts

0    Je recommande telle thérapie de telle durée av...
1                     Voici ma deuxieme recommandation
2    Le client aurait besoin de tant de séances por...
3    Je recommande un suivi sur les capacités paren...
4    Une évaluation au domicile serait nécessaire p...
dtype: object

In [34]:
keywords.loc[keywords[['depression','travail']].any(axis=1), :].index

Int64Index([0, 1, 2, 4, 5, 6], dtype='int64')

In [23]:
def get_text_list(keyword_list, keyword_dataframe, text_series):
    return text_series[keyword_dataframe.loc[keyword_dataframe[keyword_list].all(axis=1), :].index].values

In [29]:
test_list = ['travail']

In [30]:
print(get_text_list(test_list, keywords, texts))

['Le client aurait besoin de tant de séances portant sur tel enjeu'
 'Une évaluation au domicile serait nécessaire pour poursuivre le processus'
 "croyons que dû à l'épuisement profession"]


In [31]:
def simple_entry(text, keyword_list):
    add_entry(text, keyword_list, texts, keywords)

In [32]:
text_lucie = "croyons que dû à l'épuisement profession"
keywords_lucie = ['travail','burnout']
simple_entry(text_lucie, keywords_lucie)

In [46]:
keywords.columns[keywords.columns.str.startswith('d')]

Index(['depression'], dtype='object')

In [61]:
def valid_keys(partial_key, key_df=keywords):
    mask = key_df.columns.str.lower().str.startswith(partial_key.lower())
    return list(key_df.columns[mask])

In [62]:
valid_keys('t')

['travail', 'TDAH']